# SAY SOSMETHING HERE


 Your task will be to help a company operating small and mid-size apartments hosting 2-6 guests. The company is set to price their new apartments not on the market.
 
- Build a price prediction model similarly to how we did in our case study for London.
- Discuss your modeling decisions and compare your results to those of the case study.

In [1]:
#@ least 3 different models: (I have to argue for my choice of models)
#Model 1: OLS linear regression using cross-validation
#Model 2: OLS linear regression using LASSO
#Model 3: Random Forest
#Model 4: could do Gradient Boosting Machine (GBM) which is a Boosting algorithm

In [2]:
import pandas as pd

In [3]:
data = pd.read_csv('austinairbnblistings.csv')

In [4]:
data.head(3)

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license
0,5456,Guesthouse in Austin · ★4.84 · 1 bedroom · 2 b...,8028,Sylvia,NaN,78702,30.26057,-97.73441,Entire home/apt,101.0,2,668,2023-11-20,3.71,1,330,47,NaN
1,5769,Home in Austin · ★4.91 · 1 bedroom · 1 bed · 1...,8186,Elizabeth,NaN,78729,30.45697,-97.78422,Private room,NaN,1,294,2023-12-07,1.76,1,0,20,NaN
2,218402,Home in Austin · ★4.74 · 1 bedroom · 1 bed · 1...,1129520,Paulette,NaN,78754,30.36145,-97.64397,Private room,30.0,28,29,2023-09-24,0.20,1,252,2,NaN


In [5]:
data.shape

(15419, 18)

In [6]:
data = data.loc[data['room_type'] == 'Entire home/apt']

In [7]:
in_depth_data = pd.read_csv('indepthaustinairbnb.csv')

In [8]:
in_depth_data.head(3)

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,5456,https://www.airbnb.com/rooms/5456,20231215200307,2023-12-16,city scrape,Guesthouse in Austin · ★4.84 · 1 bedroom · 2 b...,NaN,My neighborhood is ideally located if you want...,https://a0.muscache.com/pictures/14084884/b5a3...,8028,...,4.83,4.73,4.79,NaN,f,1,1,0,0,3.71
1,5769,https://www.airbnb.com/rooms/5769,20231215200307,2023-12-16,previous scrape,Home in Austin · ★4.91 · 1 bedroom · 1 bed · 1...,NaN,Quiet neighborhood with lots of trees and good...,https://a0.muscache.com/pictures/23822033/ac94...,8186,...,4.94,4.76,4.92,NaN,f,1,0,1,0,1.76
2,6413,https://www.airbnb.com/rooms/6413,20231215200307,2023-12-16,previous scrape,Guesthouse in Austin · ★4.97 · Studio · 1 bed ...,NaN,Travis Heights is one of the oldest neighborho...,https://a0.muscache.com/pictures/miso/Hosting-...,13879,...,4.98,4.87,4.93,NaN,f,1,1,0,0,0.70


In [9]:
in_depth_data = in_depth_data.loc[(in_depth_data['accommodates'] >=2) & (in_depth_data['accommodates'] <=6) ]

In [10]:
in_depth_data.accommodates.value_counts()

accommodates
2    3964
4    3340
6    2293
3    1047
5     837
Name: count, dtype: int64

In [11]:
airbnb = pd.merge(data, in_depth_data, on = 'id')

In [12]:
airbnb.head(3)

,id,name_x,host_id_x,host_name_x,neighbourhood_group,neighbourhood_x,latitude_x,longitude_x,room_type_x,price_x,...,review_scores_communication,review_scores_location,review_scores_value,license_y,instant_bookable,calculated_host_listings_count_y,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month_y
0,5456,Guesthouse in Austin · ★4.84 · 1 bedroom · 2 b...,8028,Sylvia,NaN,78702,30.26057,-97.73441,Entire home/apt,101.0,...,4.83,4.73,4.79,NaN,f,1,1,0,0,3.71
1,6413,Guesthouse in Austin · ★4.97 · Studio · 1 bed ...,13879,Todd,NaN,78704,30.24885,-97.73587,Entire home/apt,NaN,...,4.98,4.87,4.93,NaN,f,1,1,0,0,0.70
2,6448,Guesthouse in Austin · ★4.97 · 1 bedroom · 2 b...,14156,Amy,NaN,78704,30.26034,-97.76487,Entire home/apt,159.0,...,4.97,4.97,4.89,NaN,t,1,1,0,0,2.09


In [13]:
airbnb.rename(columns = {'price_x':'price'}, inplace = True)

In [14]:
airbnb.columns

Index(['id', 'name_x', 'host_id_x', 'host_name_x', 'neighbourhood_group',
       'neighbourhood_x', 'latitude_x', 'longitude_x', 'room_type_x', 'price',
       'minimum_nights_x', 'number_of_reviews_x', 'last_review_x',
       'reviews_per_month_x', 'calculated_host_listings_count_x',
       'availability_365_x', 'number_of_reviews_ltm_x', 'license_x',
       'listing_url', 'scrape_id', 'last_scraped', 'source', 'name_y',
       'description', 'neighborhood_overview', 'picture_url', 'host_id_y',
       'host_url', 'host_name_y', 'host_since', 'host_location', 'host_about',
       'host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'host_is_superhost', 'host_thumbnail_url', 'host_picture_url',
       'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'neighbourhood_y',
       'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'latitude_y',
       'lo

In [29]:
airbnb['bathrooms'] = airbnb['bathrooms_text'].apply(lambda x: float(x.split()[0]) if x.split()[0].replace('.', '').isdigit() else None)
#turned observations into floats 

In [50]:
airbnb['host_is_superhost'] = airbnb['host_is_superhost'].apply(lambda x: 1 if x == 't' else 0)
#turned observations into booleans

In [58]:
airbnb.maximum_nights

0         90
1         90
2        365
3         60
4         90
        ... 
9343    1125
9344     365
9345     365
9346     365
9347     365
Name: maximum_nights, Length: 9348, dtype: int64

In [ ]:
#ONLY FOR OLS LINEAR REGRESSION (doing variable selection and interaction selection only for linear regression)
#my other models will have variables chosen on their own due to algorithms!!!
# y = price
# x = bathrooms, beds, review_scores_rating (would have to group), property_type, availability_365_x???, 
#number_of_reviews???, host_acceptance_rate??????, host_is_superhost, minimum_nights_y, maximum_nights